In [2]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from sbi.inference.base import infer
from sbi import utils as utils
from sbi import analysis as analysis
from torch.distributions import Normal

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-05 14:33:45.741827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sbi_tools import TruncatedNormal, get_SNPE_posterior, SymmetricTruncatedNormal
from simulators import Simulator
from spectralcomponents import PowerLaw, GaussianEmissionLine

In [1]:
def get_SNPE_posterior(prior, simulator, **args):
    simulator, prior = prepare_for_sbi(simulator, prior)
    inference = SNPE(prior)
    theta, x = simulate_for_sbi(simulator, proposal=prior, **args)
    density_estimator = inference.append_simulations(theta, x).train()
    posterior = inference.build_posterior(density_estimator)
    return posterior


def get_SNRE_posterior(prior, simulator, num_sims, x_o):
    # Amortized inference
    # This doesn't yet work
    inference = SNRE_C(prior)
    proposal = prior
    theta = proposal.sample((num_sims,))
    x = simulator(theta)
    _ = inference.append_simulations(theta, x).train(
        num_classes=5,
        gamma=1.0,
    )
    posterior = inference.build_posterior()
    return posterior

In [4]:
c1 = PowerLaw()
true_params = (1, 1)

prior = [SymmetricTruncatedNormal(loc=torch.tensor([0.0]), scale=torch.tensor([1.0])),
         SymmetricTruncatedNormal(loc=torch.tensor([0.0]), scale=torch.tensor([1.0])),]

def simulate_simple(params):
    simulator =  Simulator(c1, 1000, pileup='channels')
    return simulator((params))

posterior = get_SNPE_posterior(prior, simulate_simple, num_simulations=1000)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sbi/utils/user_input_checks.py:76: UserWarning: Prior was provided as a sequence of 2 priors. They will be
            interpreted as independent of each other and matched in order to the
            components of the parameter.
  warnings.warn(
Running 1000 simulations.:  61%|██████    | 607/1000 [00:45<01:04,  6.11it/s]